# Get Photos for a Specific Park

This notebook collects photos for a single park to test and perfect the photo collection process before scaling to all parks.

In [17]:
import pandas as pd
import requests
import json
import os
from dotenv import load_dotenv
from typing import List, Dict, Any

# Load environment variables
load_dotenv('.env.local')
API_KEY = os.getenv('GOOGLE_PLACES_API_KEY')

if not API_KEY:
    raise ValueError("Please set GOOGLE_PLACES_API_KEY in .env.local")

print(f"✅ API Key loaded: {API_KEY[:10]}...")

✅ API Key loaded: AIzaSyCyjZ...


In [18]:
# CONFIGURE: Set the park you want to get photos for
TARGET_PARK_ID = "ChIJHSII4d3NSYYRl6568PJp5BM"  # Urban Air Tyler
TARGET_PARK_NAME = "Urban Air Tyler"

# Or search by name instead
# TARGET_PARK_NAME = "Urban Air Trampoline and Adventure Park Tyler"
# TARGET_PARK_ID = None  # Will search for it

print(f"🎯 Target park: {TARGET_PARK_NAME}")
print(f"📍 Place ID: {TARGET_PARK_ID if TARGET_PARK_ID else 'Will search'}")

🎯 Target park: Urban Air Tyler
📍 Place ID: ChIJHSII4d3NSYYRl6568PJp5BM


In [ ]:
def get_place_details_with_photos(place_id: str) -> Dict:
    """Get detailed place information including photos"""
    url = f'https://places.googleapis.com/v1/places/{place_id}'
    
    headers = {
        'X-Goog-Api-Key': API_KEY,
        'X-Goog-FieldMask': 'id,displayName,formattedAddress,location,nationalPhoneNumber,websiteUri,regularOpeningHours,rating,userRatingCount,photos,types'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"❌ Error getting details for {place_id}: {e}")
        return {}

def check_api_permissions():
    """Check what APIs are enabled for this key"""
    print("🔍 Checking API key permissions...")
    
    # Test basic Places API access
    test_url = f'https://places.googleapis.com/v1/places/{TARGET_PARK_ID}'
    headers = {'X-Goog-Api-Key': API_KEY, 'X-Goog-FieldMask': 'id,displayName'}
    
    try:
        response = requests.get(test_url, headers=headers)
        print(f"✅ Places API v1 access: {response.status_code}")
        if response.status_code != 200:
            print(f"❌ Error: {response.text}")
    except Exception as e:
        print(f"❌ Places API error: {e}")
    
    # Test legacy Places API
    legacy_url = f'https://maps.googleapis.com/maps/api/place/details/json'
    params = {'place_id': TARGET_PARK_ID, 'key': API_KEY, 'fields': 'name,photos'}
    
    try:
        response = requests.get(legacy_url, params=params)
        print(f"✅ Legacy Places API access: {response.status_code}")
        if response.status_code == 200:
            data = response.json()
            if data.get('status') == 'OK':
                photos = data.get('result', {}).get('photos', [])
                print(f"📸 Legacy API found {len(photos)} photos")
                if photos:
                    print(f"📷 First photo reference: {photos[0].get('photo_reference', 'None')[:50]}...")
            else:
                print(f"❌ Legacy API status: {data.get('status')} - {data.get('error_message', '')}")
    except Exception as e:
        print(f"❌ Legacy Places API error: {e}")

def generate_photo_urls_diagnostic(photos: List[Dict], max_photos: int = 3) -> List[str]:
    """Generate photo URLs with detailed diagnostics"""
    if not photos:
        print("❌ No photos provided")
        return []
    
    print(f"\\n🔍 Analyzing {len(photos)} photos...")
    photo_urls = []
    
    for i, photo in enumerate(photos[:max_photos]):
        print(f"\\n📸 Photo {i+1}:")
        print(f"  Raw photo data: {json.dumps(photo, indent=2)}")
        
        photo_name = photo.get('name', '')
        if not photo_name:
            print("  ❌ No 'name' field found")
            continue
            
        print(f"  Photo name: {photo_name}")
        
        # Try the new v1 format
        photo_url_v1 = f"https://places.googleapis.com/v1/{photo_name}/media?maxWidthPx=400&key={API_KEY}"
        print(f"  Testing v1 URL: {photo_url_v1}")
        
        try:
            test_response = requests.head(photo_url_v1, timeout=10)
            print(f"  ✅ v1 Status: {test_response.status_code}")
            if test_response.status_code == 200:
                print(f"  🎉 v1 format works!")
                photo_urls.append(photo_url_v1)
                continue
            elif test_response.status_code == 403:
                print(f"  ❌ v1 403: Permission denied - check API key permissions")
            elif test_response.status_code == 404:
                print(f"  ❌ v1 404: Photo not found - URL format may be wrong")
        except Exception as e:
            print(f"  ❌ v1 Error: {e}")
        
        # If we get here, v1 didn't work
        print("  ℹ️ v1 format failed, photos may not be accessible via API")
    
    return photo_urls

# Run API diagnostics
check_api_permissions()

In [20]:
# Get place details and photos
print(f"🔍 Getting details for {TARGET_PARK_NAME}...")

if not TARGET_PARK_ID:
    # If no place ID provided, would need to search first
    print("❌ Please provide a TARGET_PARK_ID")
else:
    # Get detailed place information
    place_details = get_place_details_with_photos(TARGET_PARK_ID)
    
    if place_details:
        print(f"✅ Found place: {place_details.get('displayName', {}).get('text', 'Unknown')}")
        print(f"📍 Address: {place_details.get('formattedAddress', 'Unknown')}")
        print(f"⭐ Rating: {place_details.get('rating', 'N/A')} ({place_details.get('userRatingCount', 0)} reviews)")
        
        # Check photos
        photos = place_details.get('photos', [])
        print(f"📸 Found {len(photos)} photos")
        
        if photos:
            print("\n📷 Photo references:")
            for i, photo in enumerate(photos[:5]):  # Show first 5
                print(f"  {i+1}. {photo.get('name', 'No reference')}")
        
    else:
        print("❌ Could not get place details")

🔍 Getting details for Urban Air Tyler...
✅ Found place: Urban Air Trampoline and Adventure Park
📍 Address: 8958 S Broadway Ave, Tyler, TX 75703, USA
⭐ Rating: 4.5 (1108 reviews)
📸 Found 10 photos

📷 Photo references:
  1. places/ChIJHSII4d3NSYYRl6568PJp5BM/photos/AciIO2e9BVRR3HM22hZQSVXALZTlegjp7JbjHJvE90Y25dg5Aw3FkPeovN0W6hIaZXvQc-XV2yLyeT6SN-8td1pi3aYwaTyRoRijkOvURypNTiaJLvHRpkZ-iGqeGrcH-z8y5NtI4jecofE13u-Tf4Vl7rsGWAhsXegvUTWVOKBSeLkOnUSL-Pia5UQYlKeDHs5Ji6Fc08rBa-8i1gZLxkGjZTP7tqdYoBxwHfLgvjGEH-KV8jNJlbQZcNI3A8HUH7rPUQ7Jvv9Q_HV9WLl7GORdqgLXMPHFjZC9Nzmh4rUVLz0bCcfO2JE9V16Lkjo1DKQjydws-99qaMZiyuCUgR5adALOIPOBExX5AjJTcWZBdwi1Db1m87Ts9UbnEvAUR0PTAnfQESeHKevNHFvs04JkR91nZRzc3z7jlD9ACtt4kgQFPJ-T
  2. places/ChIJHSII4d3NSYYRl6568PJp5BM/photos/AciIO2fsxkOqy4MsdTe9CLxJoSpxtMQ0JwVhtcBRMs_P4gmU2HS9hZNvkvFcwtogaY11GYKRAYdE6rAKbTdTf0gVbE10ZQK--YoT2IvvgO8DJGb_LdZixoqwfry3-e3RWndUJPB18JGW1aUeeDQkVn0v2i606cqfnVyZL5qb7tbvZcbR7CB9SZ48DVy6kBl2xFlOiG1a4Ts9gcn7aa1QXZO44b2LcDkrAJDVbqDBCJiVUl0KHPiNH_uEOpj1

In [ ]:
# Generate photo URLs with diagnostics
if place_details and place_details.get('photos'):
    print("🖼️ Running diagnostic photo URL generation...")
    
    photo_urls = generate_photo_urls_diagnostic(place_details['photos'], max_photos=3)
    
    if photo_urls:
        print(f"\n✅ Generated {len(photo_urls)} working photo URLs")
        print("\n🔗 Working Photo URLs:")
        for i, url in enumerate(photo_urls, 1):
            print(f"  {i}. {url}")
    else:
        print("\n❌ No working photo URLs found")
        
        # Suggest alternative approaches
        print("\n💡 Alternative approaches:")
        print("1. Check Google Cloud Console API restrictions")
        print("2. Enable Photos API if available")
        print("3. Use stock photos as placeholders")
        print("4. Extract photos from park websites")
        
else:
    print("❌ No photos found for this place")

In [ ]:
# Create the updated park data with photos
if photo_urls:
    print("📝 Creating updated park data...")
    
    # Basic park info from the API response
    updated_park_data = {
        "id": TARGET_PARK_ID,
        "name": place_details.get('displayName', {}).get('text', TARGET_PARK_NAME),
        "address": place_details.get('formattedAddress', ''),
        "phone": place_details.get('nationalPhoneNumber', ''),
        "website": place_details.get('websiteUri', ''),
        "rating": place_details.get('rating'),
        "reviewCount": place_details.get('userRatingCount'),
        "location": place_details.get('location', {}),
        "photos": photo_urls,
        "photoCount": len(photo_urls)
    }
    
    print("✅ Updated park data:")
    print(json.dumps(updated_park_data, indent=2))
    
    # Save to file for easy copying
    with open('urban_air_tyler_photos.json', 'w') as f:
        json.dump(updated_park_data, f, indent=2)
    
    print("\n💾 Saved to urban_air_tyler_photos.json")
    
else:
    print("❌ No photo URLs to save")

In [ ]:
# Display the photo URLs in a format ready to copy into texas-parks.ts
if photo_urls:
    print("📋 Copy this for the 'images' field in texas-parks.ts:")
    print("\n```typescript")
    print('"images": [')
    for i, url in enumerate(photo_urls):
        comma = ',' if i < len(photo_urls) - 1 else ''
        print(f'  "{url}"{comma}')
    print('],')
    print("```")
    
    print(f"\n📊 Summary:")
    print(f"  Park: {updated_park_data['name']}")
    print(f"  Photos: {len(photo_urls)}")
    print(f"  Rating: {updated_park_data['rating']} stars")
    print(f"  Reviews: {updated_park_data['reviewCount']}")
    
    print("\n🚀 Next steps:")
    print("1. Copy the images array above")
    print("2. Paste it into the Urban Air Tyler entry in texas-parks.ts")
    print("3. Test the photo gallery on the park page")
    print("4. If it looks good, we can scale this to other parks!")
    
else:
    print("❌ No photos collected")